# CMSE 402 Pre-Class Assignment, Day07
### Mikel Good
### 1/29/2026

In [35]:
import pandas as pd
from IPython.display import HTML
import geopandas as gdf

## Exploring D3.js

In [2]:
data = pd.read_csv("acs2015_county_data.csv")

In [28]:
low_bound = sorted(data["Income"])[-10]
low_bound

98657.0

In [41]:
top_10 = data[data["Income"] >= low_bound]

In [42]:
top_10.iloc[:, :14].sort_values(by="Income", ascending=False)

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income
2872,51107,Virginia,Loudoun,351129,173986,177143,13.2,59.3,7.2,0.2,16.1,0.1,213498,123453.0
2925,51610,Virginia,Falls Church city,13308,6374,6934,9.7,72.6,2.1,0.0,11.4,0.0,9069,120522.0
2848,51059,Virginia,Fairfax,1128722,558606,570116,16.1,52.6,9.1,0.1,18.3,0.1,712214,112552.0
1205,24027,Maryland,Howard,304115,149157,154958,6.3,56.1,17.8,0.2,16.2,0.0,205466,110238.0
2826,51013,Virginia,Arlington,223945,112006,111939,15.5,63.0,8.3,0.3,9.9,0.1,159816,105763.0
1783,34019,New Jersey,Hunterdon,126250,62901,63349,5.9,86.9,2.4,0.1,3.6,0.2,94644,105444.0
2924,51600,Virginia,Fairfax city,23402,11482,11920,16.4,59.8,4.7,0.1,15.4,0.1,15638,105297.0
262,8035,Colorado,Douglas,306974,152339,154635,8.1,84.0,1.0,0.2,4.1,0.0,209369,102964.0
1810,35028,New Mexico,Los Alamos,17939,9179,8760,16.2,73.9,0.5,1.0,6.7,0.1,12955,101934.0
1791,34035,New Jersey,Somerset,330604,161252,169352,14.0,59.4,8.8,0.1,16.0,0.0,221261,100667.0


In [55]:
for i in range(len(top_10)):
    print("County: {}, frequency: {}".format(top_10.iloc[i]["County"], top_10.iloc[i]["Income"]))

County: Douglas, frequency: 102964.0
County: Howard, frequency: 110238.0
County: Montgomery, frequency: 99435.0
County: Hunterdon, frequency: 105444.0
County: Morris, frequency: 100214.0
County: Somerset, frequency: 100667.0
County: Los Alamos, frequency: 101934.0
County: Nassau, frequency: 99465.0
County: Arlington, frequency: 105763.0
County: Fairfax, frequency: 112552.0
County: Loudoun, frequency: 123453.0
County: Prince William, frequency: 98657.0
County: Fairfax city, frequency: 105297.0
County: Falls Church city, frequency: 120522.0


In [64]:
html_content = """
<!DOCTYPE html>
<meta charset="utf-8">
<div id="chart"></div>

<script src="https://d3js.org/d3.v7.min.js"></script>

<script>

// Your data
const data = [
  {County: "Douglas", frequency: 102964.0},
  {County: "Howard", frequency: 110238.0},
  {County: "Montgomery", frequency: 99435.0},
  {County: "Hunterdon", frequency: 105444.0},
  {County: "Morris", frequency: 100214.0}
];

const width = 928;
const height = 500;
const marginTop = 20;
const marginRight = 0;
const marginBottom = 30;
const marginLeft = 40;

// X scale uses County names
const x = d3.scaleBand()
    .domain(d3.sort(data, d => -d.frequency).map(d => d.County))
    .range([marginLeft, width - marginRight])
    .padding(0.1);

const xAxis = d3.axisBottom(x).tickSizeOuter(0);

// Y scale uses frequency
const y = d3.scaleLinear()
    .domain([0, d3.max(data, d => d.frequency)]).nice()
    .range([height - marginBottom, marginTop]);

// SVG container
const svg = d3.select("#chart")
    .append("svg")
    .attr("viewBox", [0, 0, width, height])
    .attr("width", width)
    .attr("height", height)
    .attr("style", "max-width: 100%; height: auto;")
    .call(zoom);

// Bars
svg.append("g")
    .attr("class", "bars")
    .attr("fill", "steelblue")
  .selectAll("rect")
  .data(data)
  .join("rect")
    .attr("x", d => x(d.County))
    .attr("y", d => y(d.frequency))
    .attr("height", d => y(0) - y(d.frequency))
    .attr("width", x.bandwidth());

// X-axis
svg.append("g")
    .attr("class", "x-axis")
    .attr("transform", `translate(0,${height - marginBottom})`)
    .call(xAxis);

// Y-axis
svg.append("g")
    .attr("class", "y-axis")
    .attr("transform", `translate(${marginLeft},0)`)
    .call(d3.axisLeft(y))
    .call(g => g.select(".domain").remove());

// Zoom behavior
function zoom(svg) {
  const extent = [[marginLeft, marginTop], [width - marginRight, height - marginTop]];

  svg.call(d3.zoom()
      .scaleExtent([1, 8])
      .translateExtent(extent)
      .extent(extent)
      .on("zoom", zoomed));

  function zoomed(event) {
    x.range([marginLeft, width - marginRight].map(d => event.transform.applyX(d)));
    svg.selectAll(".bars rect")
       .attr("x", d => x(d.County))
       .attr("width", x.bandwidth());
    svg.selectAll(".x-axis").call(xAxis);
  }
}

</script>
"""

with open("Top5_Wealthiest_Counties.html", "w") as f:
    f.write(html_content)

"Saved as Top5_Wealthiest_Counties.html"

'Saved as Top5_Wealthiest_Counties.html'

## Wrap up Questions

1. What is this tool/software package and what is it used for? (Why does it exist?)
    * D3.js is a low-level JavaScript package that is made for creating very custom visualizations. It is designed to create data-driven visualizations in the browser, and it allows fine control over how data gets portrayed visually. On top of this, it is also the building blocks for many higher-level plotting libraries.
2. What are the strengths of this tool?
   * The strengths of D3.js are that it puts all the control in the hands of the user. There isn't a single framework that the user has to follow in order to create a graph, but they are instead able to tailor it to their desires. Additionally, D3.js plots are both interactive and dynamic, so the graphical opportunities that one has with this library are endless.
3. What are its weaknesses/limitations?
    * The weaknesses of D3.js are directly tied into its strengths. Because it is so open to the unique ideas of the user, it requires a lot more code to create graphics. This is displayed in the documentation, for it can take hundreds of lines of code just to create a simple graphic. Also, it takes a lot more learning to be able to create plots than other libraries, part of the reason being that it is JavaScript-based.
4. Under what circumstances would you recommend using this?
   * I would recommend using this tool if you are in a scenario where you need a unique graphic to communicate your intended message. Just about anything you can think of, you can do with D3.js, you just have to be able to write the necessary code to execute it. 